## converting the job scheduling problem into QAOA

In [ ]:
from qiskit import Aer, execute
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit import algorithms
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA
from qiskit.utils import QuantumInstance
#from qiskit.algorithms.minimum_eigensolvers import QAOA

 
jc = 7
mc = 3
tc = 3
#Step 2: Define the Quadratic Program
# Define the Quadratic Program
qp = QuadraticProgram()

# Add binary variables for each job-timeslot combination
for i in range(1, mc+1):  # For 3 jobs
    for j in range(1, jc+1):  # For 2 timeslots
        for k in range(1,tc+1):
            qp.binary_var(f'machine{i}job{j}time{k}')

# Define the objective function (minimize the total duration)
# For simplicity, we'll assume each job has a unit duration
linear = {}
for i in range(1, mc+1):
    for j in range(1, jc+1):
        for k in range(1, tc+1):
            linear [f'machine{i}job{j}time{k}'] = 1
qp.minimize(linear=linear)


    
# 
for i in range(1, tc+1):
    job_vars = {f'machine{j}job{k}time{i}': 1 for j in range(1, mc+1) for k in range(1, jc+1)}
    rhs= max(min(jc-(mc*(i-1)), mc), 0)   
    # 7 3 7/3=2 7%3=2 i<7/3 ?
    qp.linear_constraint(linear=job_vars, sense='==', rhs= rhs)  # all variables for a machine and time must sum to 1
                    
# all jobs are ran once: for job i, sum(machine_jjob_itime_k)=duration of job i,and sum on each machine for job i except for 1 should be 0
for i in range(1, jc+1): # iterating over machines
    job_vars = {f'machine{k}job{i}time{j}': 1 for k in range(1, mc+1) for j in range(1, tc+1)}  # create a dict comprehension for all jobs
    qp.linear_constraint(linear=job_vars, sense='==', rhs=1)  # all variables for a machine and time must sum to 1

# no intersection of intervals: for machine i and time j , sum(machine_ijob_ktime_j) for all k jobs must be 1
for i in range(1, mc+1):      # iterating over machines
    for j in range(1, tc+1):  # iterating over time slots
        job_vars = {f'machine{i}job{k}time{j}': 1 for k in range(1, jc+1)}  
        qp.linear_constraint(linear=job_vars, sense='<=', rhs=1)  # all variables for a machine and time must sum to 1
    

# Set up the QAOA algorithm
qaoa = QAOA(quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))

# Use the MinimumEigenOptimizer to solve the QUBO with QAOA
optimizer = MinimumEigenOptimizer(qaoa)
result = optimizer.solve(qp)

print(result)




/tmp/ipykernel_45060/335316709.py:58: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qaoa = QAOA(quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))
/tmp/ipykernel_45060/335316709.py:58: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))
